In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("proj1.sqlite")
cur = conn.cursor()


This querry lists the names of all the unique clients.

In [4]:
query = "select name from person_corporation where person_id in (select client_id from clients);"
df = pd.read_sql_query(query, conn)
df

,name
0,Ximena Gardner
1,Rebekah Hart
2,Anne Costa
3,Lyric Dennis
4,Hope Campbell
5,Ally Lambert
6,Seamus Paul
7,Karla Decker
8,Veronica Dillon
9,Jamarcus Callahan


This query finds the unique names of owners and total square footage of all the properties they own

In [5]:
query = "select owner_id, sum(area) from (select owner_id, area from owns, properties where owns.property_number =  properties.property_number) as owner_area group by owner_id;"
df = pd.read_sql_query(query, conn)
df


,owner_id,sum(area)
0,19,1200
1,20,2200
2,21,1200
3,22,3000
4,23,2400
5,24,12000
6,25,24000


This query finds the properties shown by each associate in a given month. 

In [12]:
year = int(input("please input the year: "))
month = int(input("please input the month(in number): "))
lower = (year*10000) + (month*100)
upper = (year*10000) + ((month + 1)*100)
query = "select name, viewing_date.property_number from viewing_date, (select employe_id, name, property_number from associate, person_corporation where associate.employe_id = person_corporation.person_id) as assos where viewing_date.property_number = assos.property_number and date_ >= " + str(lower) + " and date_ < " + str(upper) + ";"

df = pd.read_sql_query(query, conn)
df


please input the year: 2020
please input the month(in number): 12


,name,property_number
0,Bailey Larson,1
1,Bailey Larson,1
2,Bailey Larson,2
3,Bailey Larson,2
4,Halle Oconnell,3
5,Halle Oconnell,3
6,Halle Oconnell,4
7,Halle Oconnell,4
8,Naomi Mcconnell,5
9,Naomi Mcconnell,5


This query finds the most popular properties (in terms of number of viewings in a given year). 

In [23]:
query = "select  property_number, count(property_number) as amount_viewed from viewing_date where date_ >= 20200000 and date_ < 20210000 group by property_number order by amount_viewed desc;"

df = pd.read_sql_query(query, conn)
df


,property_number,amount_viewed
0,6,2
1,5,2
2,4,2
3,3,2
4,2,2
5,1,2


This query finds the total rent due to each property owner.

In [15]:
query = "select owner_id, name, sum(rent) as total_rent from (select owns.owner_id,  person_corporation.name, owns.property_number,  lease_rent.rent from person_corporation, owns , (select rent, property_number from lease) lease_rent where owns.property_number = lease_rent.property_number and owns.owner_id = person_corporation.person_id) as full_list group by owner_id;"

df = pd.read_sql_query(query, conn)
df


,owner_id,name,total_rent
0,19,Sofia Richard,2000
1,20,Brodie West,3500
2,21,Ana Benton,3000
3,22,Noelle Gentry,3000
4,23,Deacon Mcmahon,5000
5,24,Immanuel Holt,6500


This query finds the unique names of associates supervised (directly or indirectly) by a given employee. 

In [18]:
name = input("please input the supervisors name ")
query = "select supervisor.subordinate_id, person_corporation.name from person_corporation, supervisor where supervisor_id in (select person_id from person_corporation where name = \"" + name + "\") and supervisor.subordinate_id = person_corporation.person_id;"

df = pd.read_sql_query(query, conn)
df


please input the supervisors name Ulises Huerta


,subordinate_id,name
0,2,Roselyn Holder
1,4,Bailey Larson


This query finds the unique names of owners that have a residential property in every city where Pat Doe
owns a commercial property

In [19]:
query = "select person_corporation.name from person_corporation, owns, properties, address where person_corporation.person_id = owns.owner_id and owns.property_number = properties.property_number and properties.property_type = \"residential\" and properties.address_id = address.address_id and address.city in (select address.city from person_corporation, owns, properties, address  where person_corporation.person_id = owns.owner_id and owns.property_number = properties.property_number and person_corporation.name = \"Pat Doe\" and properties.address_id = address.address_id and properties.property_type = \"commercial\");"
df = pd.read_sql_query(query, conn)
df


,name
0,Sofia Richard


This query finds the top-3 partners with respect to number of properties leased in the current year.

In [24]:
query = "select partner.employe_id, person_corporation.name, count(lease.property_number) as lesed_property from partner, owns, lease, owner, person_corporation where partner.employe_id = owner.partner_id and owns.owner_id = owner.owner_id and owns.property_number = lease.property_number and person_corporation.person_id = partner.employe_id group by employe_id order by lesed_property desc limit 3;"
df = pd.read_sql_query(query, conn)
df


,employe_id,name,lesed_property
0,3,Brody Wagner,2
1,2,Roselyn Holder,2
2,1,Ulises Huerta,2
